In [1]:
import time
import cvxpy
import random
import numpy as np
import pandas as pd

In [7]:
# サンプルデータの読み込み
sample = pd.read_csv('sample_10*100.csv', sep='\t')

In [8]:
sample.head(3)

,id,price_A,price_B,price_C,price_D,price_E,price_F,price_G,price_H,price_I,...,rating_A,rating_B,rating_C,rating_D,rating_E,rating_F,rating_G,rating_H,rating_I,rating_J
0,1,380,385,389,375,387,385,382,378,385,...,4.1,3.3,4.0,3.5,3.8,4.5,4.1,3.8,3.8,3.8
1,2,943,947,961,960,960,947,950,940,944,...,10.1,8.9,9.3,10.3,9.4,9.5,9.9,9.4,8.4,9.1
2,3,980,980,987,982,974,982,981,982,975,...,10.4,8.7,10.3,9.8,9.3,9.0,9.8,9.8,9.8,9.8


In [14]:
# クライアトa, b, cの元々持っていた枠をランダムに決める。
length = len(sample) #dfの長さ。
ids = list(range(length)) #「0~length-1」の数字の列。

# idは1からなので、1を足す。
random_ids = [id+1 for id in ids]

# Break point.
bp1 = length//10 
bp2 = length//10 + bp1
bp3 = length//10 + bp2
bp4 = length//10 + bp3
bp5 = length//10 + bp4
bp6 = length//10 + bp5
bp7 = length//10 + bp6
bp8 = length//10 + bp7
bp9 = length//10 + bp8

# 順番シャッフルして、Break pointで切り分ける。
random.shuffle(random_ids)

id_A = random_ids[:bp1]
id_B = random_ids[bp1:bp2]
id_C = random_ids[bp2:bp3]
id_D = random_ids[bp3:bp4]
id_E = random_ids[bp4:bp5]
id_F = random_ids[bp5:bp6]
id_G = random_ids[bp6:bp7]
id_H = random_ids[bp7:bp8]
id_I = random_ids[bp8:bp9]
id_J = random_ids[bp9:]

In [15]:
# それぞれのidを表示。 ランダムにシャッフルされていることがわかる。
print(id_A)
print(id_B)
print(id_C)
print(id_D)
print(id_E)
print(id_F)
print(id_G)
print(id_H)
print(id_I)
print(id_J)

[98, 11, 37, 16, 85, 52, 83, 78, 24, 29]
[60, 90, 58, 18, 35, 63, 57, 49, 64, 4]
[28, 23, 44, 51, 50, 26, 15, 14, 46, 12]
[82, 17, 95, 79, 8, 86, 42, 21, 94, 62]
[19, 56, 48, 34, 1, 36, 76, 70, 71, 72]
[59, 53, 25, 89, 47, 87, 30, 43, 41, 10]
[32, 55, 22, 84, 38, 69, 5, 81, 91, 67]
[31, 77, 9, 93, 88, 97, 92, 66, 80, 33]
[68, 74, 73, 99, 3, 27, 20, 96, 13, 54]
[40, 75, 61, 6, 7, 65, 100, 39, 2, 45]


In [16]:
# それぞれのidのデータのみを取り出すための判別用カラム。
sample['judge_A'] = sample['id'].apply(lambda x:x in id_A)
sample['judge_B'] = sample['id'].apply(lambda x:x in id_B)
sample['judge_C'] = sample['id'].apply(lambda x:x in id_C)
sample['judge_D'] = sample['id'].apply(lambda x:x in id_D)
sample['judge_E'] = sample['id'].apply(lambda x:x in id_E)
sample['judge_F'] = sample['id'].apply(lambda x:x in id_F)
sample['judge_G'] = sample['id'].apply(lambda x:x in id_G)
sample['judge_H'] = sample['id'].apply(lambda x:x in id_H)
sample['judge_I'] = sample['id'].apply(lambda x:x in id_I)
sample['judge_J'] = sample['id'].apply(lambda x:x in id_J)

In [17]:
# それぞれのidのデータを取得。
df_A = sample[sample['judge_A']].loc[:, ['id','price_A', 'rating_A']]
df_B = sample[sample['judge_B']].loc[:, ['id','price_B', 'rating_B']]
df_C = sample[sample['judge_C']].loc[:, ['id','price_C', 'rating_C']]
df_D = sample[sample['judge_D']].loc[:, ['id','price_D', 'rating_D']]
df_E = sample[sample['judge_E']].loc[:, ['id','price_E', 'rating_E']]
df_F = sample[sample['judge_F']].loc[:, ['id','price_F', 'rating_F']]
df_G = sample[sample['judge_G']].loc[:, ['id','price_G', 'rating_G']]
df_H = sample[sample['judge_H']].loc[:, ['id','price_H', 'rating_H']]
df_I = sample[sample['judge_I']].loc[:, ['id','price_I', 'rating_I']]
df_J = sample[sample['judge_J']].loc[:, ['id','price_J', 'rating_J']]

In [18]:
# 判別用カラムを削除。
del sample['judge_A']
del sample['judge_B']
del sample['judge_C']
del sample['judge_D']
del sample['judge_E']
del sample['judge_F']
del sample['judge_G']
del sample['judge_H']
del sample['judge_I']
del sample['judge_J']

In [19]:
# 元々の値段の総和、元々の視聴率の和を計算する。
# Original price, Original point
original_price_A, original_point_A = sum(df_A['price_A']), sum(df_A['rating_A'])
original_price_B, original_point_B = sum(df_B['price_B']), sum(df_B['rating_B'])
original_price_C, original_point_C = sum(df_C['price_C']), sum(df_C['rating_C'])
original_price_D, original_point_D = sum(df_D['price_D']), sum(df_D['rating_D'])
original_price_E, original_point_E = sum(df_E['price_E']), sum(df_E['rating_E'])
original_price_F, original_point_F = sum(df_F['price_F']), sum(df_F['rating_F'])
original_price_G, original_point_G = sum(df_G['price_G']), sum(df_G['rating_G'])
original_price_H, original_point_H = sum(df_H['price_H']), sum(df_H['rating_H'])
original_price_I, original_point_I = sum(df_I['price_I']), sum(df_I['rating_I'])
original_price_J, original_point_J = sum(df_J['price_J']), sum(df_J['rating_J'])

In [20]:
# 各クライアントの考える価値（指定視聴率/値段）を数値化し、表に加える。
# 指定した視聴率/そのクライアントの号数価格
sample['value_A'] = round((sample['rating_A']/sample['price_A'])*1000, 2)
sample['value_B'] = round((sample['rating_B']/sample['price_B'])*1000, 2)
sample['value_C'] = round((sample['rating_C']/sample['price_C'])*1000, 2)
sample['value_D'] = round((sample['rating_D']/sample['price_D'])*1000, 2)
sample['value_E'] = round((sample['rating_E']/sample['price_E'])*1000, 2)
sample['value_F'] = round((sample['rating_F']/sample['price_F'])*1000, 2)
sample['value_G'] = round((sample['rating_G']/sample['price_G'])*1000, 2)
sample['value_H'] = round((sample['rating_H']/sample['price_H'])*1000, 2)
sample['value_I'] = round((sample['rating_I']/sample['price_I'])*1000, 2)
sample['value_J'] = round((sample['rating_J']/sample['price_J'])*1000, 2)

In [21]:
print('client_Aの交換前の合計価格:', original_price_A, 'client_Aの交換前の合計視聴率', original_point_A)
print('client_Bの交換前の合計価格:', original_price_B, 'client_Bの交換前の合計視聴率', original_point_B)
print('client_Cの交換前の合計価格:', original_price_C, 'client_Cの交換前の合計視聴率', original_point_C)
print('client_Dの交換前の合計価格:', original_price_D, 'client_Dの交換前の合計視聴率', original_point_D)
print('client_Eの交換前の合計価格:', original_price_E, 'client_Eの交換前の合計視聴率', original_point_E)
print('client_Fの交換前の合計価格:', original_price_F, 'client_Fの交換前の合計視聴率', original_point_F)
print('client_Gの交換前の合計価格:', original_price_G, 'client_Gの交換前の合計視聴率', original_point_G)
print('client_Hの交換前の合計価格:', original_price_H, 'client_Hの交換前の合計視聴率', original_point_H)
print('client_Iの交換前の合計価格:', original_price_I, 'client_Iの交換前の合計視聴率', original_point_I)
print('client_Jの交換前の合計価格:', original_price_J, 'client_Jの交換前の合計視聴率', original_point_J)

sample.head(3)

client_Aの交換前の合計価格: 9146 client_Aの交換前の合計視聴率 89.0
client_Bの交換前の合計価格: 6526 client_Bの交換前の合計視聴率 66.60000000000001
client_Cの交換前の合計価格: 7672 client_Cの交換前の合計視聴率 76.8
client_Dの交換前の合計価格: 9592 client_Dの交換前の合計視聴率 96.49999999999999
client_Eの交換前の合計価格: 8022 client_Eの交換前の合計視聴率 80.69999999999999
client_Fの交換前の合計価格: 7905 client_Fの交換前の合計視聴率 78.7
client_Gの交換前の合計価格: 5969 client_Gの交換前の合計視聴率 57.300000000000004
client_Hの交換前の合計価格: 10487 client_Hの交換前の合計視聴率 107.00000000000001
client_Iの交換前の合計価格: 8146 client_Iの交換前の合計視聴率 80.7
client_Jの交換前の合計価格: 7361 client_Jの交換前の合計視聴率 74.6


,id,price_A,price_B,price_C,price_D,price_E,price_F,price_G,price_H,price_I,...,value_A,value_B,value_C,value_D,value_E,value_F,value_G,value_H,value_I,value_J
0,1,380,385,389,375,387,385,382,378,385,...,10.79,8.57,10.28,9.33,9.82,11.69,10.73,10.05,9.87,9.95
1,2,943,947,961,960,960,947,950,940,944,...,10.71,9.40,9.68,10.73,9.79,10.03,10.42,10.00,8.90,9.50
2,3,980,980,987,982,974,982,981,982,975,...,10.61,8.88,10.44,9.98,9.55,9.16,9.99,9.98,10.05,9.99


In [22]:
# 交換前のデータをまとめる。
original_prices = [original_price_A, original_price_B, original_price_C, original_price_D, original_price_E,
                   original_price_F, original_price_G, original_price_H, original_price_I, original_price_J]
original_points = [original_point_A, original_point_B, original_point_C, original_point_D, original_point_E,
                   original_point_F, original_point_G, original_point_H, original_point_I, original_point_J]

In [23]:
print(original_prices)
print(original_points)

[9146, 6526, 7672, 9592, 8022, 7905, 5969, 10487, 8146, 7361]
[89.0, 66.60000000000001, 76.8, 96.49999999999999, 80.69999999999999, 78.7, 57.300000000000004, 107.00000000000001, 80.7, 74.6]


<b><font color='Red'><font size=4>※valueは、正の数じゃないと、選ばなくなってしまうことに注意。<br>（０の方が大きいから。）</font></font></b>

***
***

### 〜ここからアルゴリズム〜

In [24]:
# ナップザック問題。

# size = 値段のデータ。
# weight = 価値のデータ。
# capacity = 元々の値段×1.05倍の値段。
# ids = 取得すべきだと考えられた枠のid    

def Knapsack(size, weight, capacity, obtained=0, rate=0.05):
    # 価格が元の価格の1.05%を超えても、終わり。
    if capacity*(1 + rate) - obtained > 0:        
        x = cvxpy.Variable(size.shape[0], boolean=True)
        # 要素の個数(これを加えて平均を最大化しないと、価格の小さいものばかりをとってしまう。)
        count = sum(np.ones(len(size)) * x)
        # 目的
        objective = cvxpy.Maximize(weight * x / count)
        # 制限。ここでは、capacity(価格)の上下5%を取っている。
        constraints = [capacity*(1 + rate) -  obtained >= size * x]
        constraints += [capacity*(1 - rate) - obtained <= size * x]

        prob = cvxpy.Problem(objective, constraints)
        prob.solve(solver=cvxpy.ECOS_BB)
        result = [round(ix, 0) for ix in x.value]

        return result
    else:
        return []
    
# 2重のリストをフラットにする関数(重複は残る！)
def Flatten_dual(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

In [33]:
# dfは、上記のsampleのような、カラム名がprice_A, rating_A, value_A, price_B, ...となっているDataFrame。
# namesには、'A', 'B', 'C', ... を入れるイメージ。

def Loop(df, original_prices, original_points, *names):
    result = dict() #結果を格納する。
    num = len(names) #クライアントの数。

    obtained_prices = [0 for i in range(num)] #獲得した枠の値段をそれぞれ格納する。
    obtained_points = [0 for i in range(num)] #獲得した枠の視聴率をそれぞれ格納する。
    obtained_idses = [[] for i in range(num)] #獲得した枠のidをそれぞれ格納する。
    idses = [[] for i in range(num)] # 各ナップザック問題でそれぞれが指定した枠のidを格納する。
    add_idses = [[] for i in range(num)] # 各ナップザック問題の結果、獲得した枠のidを格納する。
    check_point = 0
    
    while check_point==0:
        for i in range(num): #それぞれのクライアントごと。
            original_price = original_prices[i] # 交換前の価格合計。
            original_point = original_points[i] # 交換前の視聴率合計。

            name = names[i] # クライアントの名前。
            size = np.array(df['price_' + name]) # dfのカラム名は、こうなるように指定。全ての枠の値段を取得。
            weight = np.array(df['value_' + name]) # dfのカラム名は、こうなるように指定。全ての枠の価値を取得。
            capacity = original_price 

            if obtained_points[i] < original_point: # 獲得した視聴率和が元々の視聴率和より低かったら
                idses[i] = Knapsack(size, weight, capacity,  obtained_prices[i], rate=0.1)
            else:
                idses[i] = [0 for i in range(len(size))] # 視聴率和が元々の視聴率和を超えていた場合、どのidも選択しない。

        if sum(Flatten_dual(idses)) == 0: #もし、どのクライアントも選んでいなかったら
            idses[0][0] = 1 # 始めのクライアントがdfの一番上の行を選んだことにする。（暫定的なエラー処理。）
            
        ids = np.array(df['id'].index) + 1

        # それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
        ids_dict = dict()
        for key in ids-1: # それぞれのidごとに、
            values = [] # そのidの枠を指定したクライアントを格納するリスト。
            for i in range(num): # それぞれのクライアントごとに、
                if key in idses[i] * ids -1: # そのid（のindex）があれば、
                    values.append(names[i]) # valuesにクライアントの名前が格納される。
            if values: # そのidを指定したクライアントがいれば、
                value = random.choice(values) # その中からクライアントをランダムに選ぶ。
                ids_dict[key] = value # idと合わせて格納される。

        for i in range(num):
            add_idses[i] = [ids for ids, client in ids_dict.items() if client == names[i]] # クライアントごとに、獲得したid
            # obtained_idses[i] += add_idses[i]
            obtained_idses[i] += df.query('index in ' + str(add_idses[i]))['id'].values.tolist()
            obtained_prices[i] += sum(df.query('index in ' + str(add_idses[i]))['price_' + name])
            obtained_points[i] += sum(df.query('index in ' + str(add_idses[i]))['rating_' + name])

        extraction_ids = Flatten_dual(add_idses) # 今回獲得されたidの集合を、取り除く。
        df = df.query('index not in ' + str(extraction_ids)).reset_index(drop=True)
        
        if len(df) == 1 and df['id'][0] != 0: #dfの長さが１だとうまく機能しないため、長さが１の場合は加える。
            add_df = pd.DataFrame(np.zeros(df.shape[1])).T # np.zeros()の中の数字は、列の数。
            add_df.columns = df.columns # カラム名を揃える。
            df = pd.concat([df, add_df]).reset_index(drop=True)

        # 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
        if len(df) <= 1:
            check_point += 1
    
    if obtained_points > original_points:
        sum_point = round(sum(obtained_points), 2) # 合計値を小数第二位までで表示。
        result[sum_point] = obtained_prices, obtained_idses
                                  
    return result

In [58]:
# 1回のループの結果。
# ちなみに、横に並べられるリストの要素数の上限は20なので、20を越えると縦に並び始める。
start = time.time()
result = Loop(sample, original_prices, original_points, 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J')
end = time.time()
print('処理時間は:', round(end-start, 3), '[sec]')
result

処理時間は: 6.188 [sec]


{809.1: ([11681, 6179.0, 6946, 8684, 7237, 7432, 6161, 11338, 8439, 6656],
  [[45, 72, 17, 47, 67, 71, 84, 83, 1, 4, 5, 10, 15, 16, 18, 20, 22, 23, 38],
   [51, 79, 12, 33, 53, 69, 88, 46, 55.0],
   [37, 42, 50, 90, 34, 36, 76],
   [81, 77, 96, 2, 31, 21, 60, 85, 91],
   [86, 89, 14, 70, 43, 8, 49, 6],
   [54, 64, 3, 9, 99, 26, 27, 35, 87],
   [52, 11, 62, 92, 39, 40, 57, 63, 94],
   [82, 65, 98, 29, 66, 32, 56, 78, 58, 59, 61, 68, 73],
   [80, 28, 41, 48, 93, 75, 19, 24, 95],
   [97, 74, 7, 13, 25, 30, 44, 100]])}

### これを何度も繰り返すことにより、resultに値を格納しまくる。

In [61]:
start = time.time()

results = {}
for i in range(10):
    results.update(Loop(sample, original_prices, original_points, 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'))

    end = time.time()
print('処理時間は:', round(end-start, 3), '[sec]')

処理時間は: 63.519 [sec]


<b>〜とりあえずループ処理10回、というやや適当（だが、結構収束している）な回数だが、処理時間はおよそ１分間〜</b>

In [62]:
# それぞれのループごとの交換の結果。
results

{809.1: ([11755, 5889, 7642.0, 9035, 7282, 8400, 5377, 11201, 7341, 6831],
  [[97,
    77,
    17,
    90,
    21,
    48,
    75,
    94,
    1,
    4,
    5,
    6,
    10,
    15,
    16,
    20,
    23,
    46,
    55,
    58],
   [37, 28, 41, 60, 11, 38],
   [86, 29, 3, 34, 78, 84, 87, 27, 69, 76.0],
   [54, 65, 96, 2, 31, 32, 7, 62, 95],
   [74, 71, 43, 93, 12, 8, 36, 53],
   [80, 50, 79, 33, 85, 30, 88, 40, 57, 63, 100],
   [81, 56, 70, 92, 35, 39, 22],
   [45, 72, 82, 64, 67, 9, 13, 19, 24, 91, 59, 61, 68, 73, 83],
   [52, 89, 98, 42, 66, 18],
   [51, 47, 14, 99, 26, 49, 25, 44]])}

In [63]:
# 最も視聴率和を大きくした交換の仕方を求める。
print('交換後の価格:',results[max(results.keys())][0])
print('交換後のそれぞれの枠')
print('A:', results[max(results.keys())][1][0])
print('B:', results[max(results.keys())][1][1])
print('C:', results[max(results.keys())][1][2])
print('D:', results[max(results.keys())][1][3])
print('E:', results[max(results.keys())][1][4])
print('F:', results[max(results.keys())][1][5])
print('G:', results[max(results.keys())][1][6])
print('H:', results[max(results.keys())][1][7])
print('I:', results[max(results.keys())][1][8])
print('J:', results[max(results.keys())][1][9])

交換後の価格: [11755, 5889, 7642.0, 9035, 7282, 8400, 5377, 11201, 7341, 6831]
交換後のそれぞれの枠
A: [97, 77, 17, 90, 21, 48, 75, 94, 1, 4, 5, 6, 10, 15, 16, 20, 23, 46, 55, 58]
B: [37, 28, 41, 60, 11, 38]
C: [86, 29, 3, 34, 78, 84, 87, 27, 69, 76.0]
D: [54, 65, 96, 2, 31, 32, 7, 62, 95]
E: [74, 71, 43, 93, 12, 8, 36, 53]
F: [80, 50, 79, 33, 85, 30, 88, 40, 57, 63, 100]
G: [81, 56, 70, 92, 35, 39, 22]
H: [45, 72, 82, 64, 67, 9, 13, 19, 24, 91, 59, 61, 68, 73, 83]
I: [52, 89, 98, 42, 66, 18]
J: [51, 47, 14, 99, 26, 49, 25, 44]


<b><font size=4><font color='#113BC5'>この数（クライアントの数：１０ 枠の数：１００）でも、思いの外時間がかかる…。</font></font></b>